<a href="https://colab.research.google.com/github/eghib22/Store-Sales-Forecasting/blob/main/model_experiment_N_BEATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle
! pip install wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! mkdir ~/.kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ekaterineghibradze","key":"b1414052fbae86987efff2083c8dcbd1"}'}

In [5]:
!mv "kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!ls -l ~/.kaggle/

total 4
-rw------- 1 root root 74 Jul  7 22:52 kaggle.json


In [7]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 367MB/s]


In [8]:
! unzip walmart-recruiting-store-sales-forecasting

Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [9]:
!unzip '*.csv.zip'

Archive:  features.csv.zip
  inflating: features.csv            

Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  test.csv.zip
  inflating: test.csv                

4 archives were successfully processed.


In [10]:
!unzip '*.csv.zip'

Archive:  features.csv.zip
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv            

Archive:  sampleSubmission.csv.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               

Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                

4 archives were successfully processed.


In [11]:
import pandas as pd

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')
sample_submission = pd.read_csv('sampleSubmission.csv')

# Explore the data
print("Train shape:", train.shape)
print(train.head())
print(train.columns)

print("\nTest shape:", test.shape)
print(test.head())
print(test.columns)

print("\nFeatures shape:", features.shape)
print(features.head())
print(features.columns)

print("\nStores shape:", stores.shape)
print(stores.head())
print(stores.columns)

print("\nSample Submission shape:", sample_submission.shape)
print(sample_submission.head())
print(sample_submission.columns)

Train shape: (421570, 5)
   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday'], dtype='object')

Test shape: (115064, 4)
   Store  Dept        Date  IsHoliday
0      1     1  2012-11-02      False
1      1     1  2012-11-09      False
2      1     1  2012-11-16      False
3      1     1  2012-11-23       True
4      1     1  2012-11-30      False
Index(['Store', 'Dept', 'Date', 'IsHoliday'], dtype='object')

Features shape: (8190, 12)
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        3

In [12]:
# Merge train and test with features and stores
train_merged = pd.merge(train, features, on=['Store', 'Date'], how='left')
train_merged = pd.merge(train_merged, stores, on='Store', how='left')

test_merged = pd.merge(test, features, on=['Store', 'Date'], how='left')
test_merged = pd.merge(test_merged, stores, on='Store', how='left')

# Convert Date to datetime
train_merged['Date'] = pd.to_datetime(train_merged['Date'])


In [13]:
# Split
train_data = train_merged[train_merged['Date'] < '2012-01-01']
val_data = train_merged[(train_merged['Date'] >= '2012-01-01') & (train_merged['Date'] < '2012-07-01')]
test_data = train_merged[train_merged['Date'] >= '2012-07-01']

print("Train:", train_data.shape)
print("Validation:", val_data.shape)
print("Test (local):", test_data.shape)

Train: (294132, 17)
Validation: (77110, 17)
Test (local): (50328, 17)


In [14]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eghib22 (eghib22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [15]:
def preprocess(df):
    df = df.copy()

    # Map Type A/B/C to 0/1/2
    type_map = {'A': 0, 'B': 1, 'C': 2}
    df['Type'] = df['Type'].map(type_map)

    # Ensure IsHoliday is int (True/False → 1/0)
    if 'IsHoliday_x' in df.columns:
        df['IsHoliday'] = df['IsHoliday_x'].astype(int)
        df = df.drop(columns=['IsHoliday_x', 'IsHoliday_y'])
    elif 'IsHoliday' in df.columns:
        df['IsHoliday'] = df['IsHoliday'].astype(int)

    # Fill NaNs in MarkDown columns with 0
    markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
    for col in markdown_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    # Extract date features
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
    df['Day'] = df['Date'].dt.day

    return df


In [18]:
# Optimized WandB configuration
wandb_config = {
    "epochs": 100,  # Increased from 50
    "batch_size": 64,  # Increased from 32 for better gradient estimates
    "learning_rate": 0.0005,  # Reduced from 0.001
    "lookback": 16,  # Increased from 12 for more context
    "horizon": 1,
    "num_blocks": 2,  # Reduced from 3 to prevent overfitting
    "units": 128,  # Reduced from 256 to prevent overfitting
    "dropout_rate": 0.3,  # Increased regularization
    "model_type": "N-BEATS",
    "dataset": "Walmart Sales",
    "optimizer": "Adam",
    "loss": "MSE",
    "weight_decay": 1e-5,  # L2 regularization
    "gradient_clip_norm": 1.0  # Gradient clipping
}

def create_walmart_nbeats_model(input_shape, horizon=1, num_blocks=2, units=128, dropout_rate=0.3):
    """Create optimized N-BEATS inspired model"""

    # Input layer
    inputs = layers.Input(shape=input_shape, name='input_layer')

    # Flatten the input
    flattened = layers.Flatten(name='flatten')(inputs)

    # Improved preprocessing with batch normalization
    x = layers.Dense(256, activation='relu', name='dense_preprocess_1')(flattened)
    x = layers.BatchNormalization(name='bn_1')(x)
    x = layers.Dropout(dropout_rate, name='dropout_1')(x)

    x = layers.Dense(128, activation='relu', name='dense_preprocess_2')(x)
    x = layers.BatchNormalization(name='bn_2')(x)
    x = layers.Dropout(dropout_rate, name='dropout_2')(x)

    # N-BEATS blocks with improved architecture
    forecasts = []
    residual = x

    for i in range(num_blocks):
        # Block with residual connections and batch normalization
        block_out = layers.Dense(units, activation='relu', name=f'block_{i}_dense_1')(residual)
        block_out = layers.BatchNormalization(name=f'block_{i}_bn_1')(block_out)
        block_out = layers.Dropout(dropout_rate, name=f'block_{i}_dropout_1')(block_out)

        block_out = layers.Dense(units, activation='relu', name=f'block_{i}_dense_2')(block_out)
        block_out = layers.BatchNormalization(name=f'block_{i}_bn_2')(block_out)
        block_out = layers.Dropout(dropout_rate, name=f'block_{i}_dropout_2')(block_out)

        block_out = layers.Dense(units//2, activation='relu', name=f'block_{i}_dense_3')(block_out)
        block_out = layers.BatchNormalization(name=f'block_{i}_bn_3')(block_out)

        # Forecast and backcast with proper dimensions
        forecast = layers.Dense(horizon, activation='linear', name=f'forecast_{i}')(block_out)
        backcast = layers.Dense(units//2, activation='linear', name=f'backcast_{i}')(block_out)

        forecasts.append(forecast)

        # Proper residual connection
        if residual.shape[-1] != backcast.shape[-1]:
            residual = layers.Dense(units//2, name=f'residual_proj_{i}')(residual)

        residual = layers.Subtract(name=f'residual_subtract_{i}')([residual, backcast])

    # Sum all forecasts
    if len(forecasts) > 1:
        output = layers.Add(name='sum_forecasts')(forecasts)
    else:
        output = forecasts[0]

    # Final output with regularization
    output = layers.Dense(horizon//2 if horizon > 1 else 32, activation='relu', name='final_dense')(output)
    output = layers.Dropout(dropout_rate/2, name='final_dropout')(output)
    output = layers.Dense(horizon, activation='linear', name='final_output')(output)

    model = keras.Model(inputs=inputs, outputs=output, name='walmart_nbeats_optimized')
    return model

def train_nbeats_model_optimized(X_train, y_train, X_val, y_val,
                                epochs=100, batch_size=64, learning_rate=0.0005):
    """Train optimized N-BEATS model"""

    # Create model
    model = create_walmart_nbeats_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        horizon=y_train.shape[1],
        num_blocks=wandb_config["num_blocks"],
        units=wandb_config["units"],
        dropout_rate=wandb_config["dropout_rate"]
    )

    # Print model summary
    print("Optimized Model Summary:")
    model.summary()

    # Compile with improved optimizer settings
    optimizer = keras.optimizers.Adam(
        learning_rate=learning_rate,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-7,
        weight_decay=wandb_config.get("weight_decay", 1e-5)
    )

    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae', 'mape']
    )

    # Improved callbacks
    callbacks = [
        # More patient early stopping
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=50,  # Increased from 10
            restore_best_weights=True,
            verbose=1,
            min_delta=1e-6  # Minimum improvement threshold
        ),
        # More conservative learning rate reduction
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.7,  # Less aggressive reduction
            patience=10,  # Increased patience
            min_lr=1e-7,  # Lower minimum LR
            verbose=1,
            min_delta=1e-6
        ),
        # Learning rate scheduling
        keras.callbacks.LearningRateScheduler(
            lambda epoch: learning_rate * (0.95 ** epoch),
            verbose=0
        ),
        # Gradient clipping callback
        keras.callbacks.LambdaCallback(
            on_batch_end=lambda batch, logs: None
        ),
        WandbMetricsLogger(log_freq="epoch"),
        WandbModelCheckpoint(
            filepath="best_model_optimized.h5",
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=False
        )
    ]

    # Train with improved settings
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1,
        shuffle=True  # Important for time series
    )

    return model, history

# Alternative: Cyclical Learning Rate
def create_cyclical_lr_callback(base_lr=1e-5, max_lr=1e-3, step_size=10):
    """Create cyclical learning rate callback"""
    def cyclical_lr(epoch):
        cycle = np.floor(1 + epoch / (2 * step_size))
        x = np.abs(epoch / step_size - 2 * cycle + 1)
        return base_lr + (max_lr - base_lr) * np.maximum(0, (1 - x))

    return keras.callbacks.LearningRateScheduler(cyclical_lr)

# Improved data preparation with better validation split
def prepare_walmart_data_optimized(train_data, val_data, test_data, lookback=16, horizon=1):
    """Improved data preparation with better temporal splits"""

    # Preprocess all datasets
    train_processed = preprocess(train_data.copy())
    val_processed = preprocess(val_data.copy())
    test_processed = preprocess(test_data.copy())

    # Sort by available columns for temporal order
    sort_cols = ['Store', 'Dept']
    if 'Date' in train_processed.columns:
        sort_cols.append('Date')
    elif 'Week' in train_processed.columns:
        sort_cols.append('Week')
    elif 'Year' in train_processed.columns and 'Month' in train_processed.columns:
        sort_cols.extend(['Year', 'Month'])

    print(f"Sorting by columns: {sort_cols}")

    train_processed = train_processed.sort_values(sort_cols)
    val_processed = val_processed.sort_values(sort_cols)
    test_processed = test_processed.sort_values(sort_cols)

    # Feature engineering improvements
    exclude_cols = ['Weekly_Sales', 'Date'] if 'Date' in train_processed.columns else ['Weekly_Sales']
    feature_cols = [col for col in train_processed.columns
                   if col not in exclude_cols and
                   train_processed[col].dtype in ['int64', 'float64', 'int32', 'float32']]

    print(f"Initial feature columns: {feature_cols}")

    # Add lag features - only if we have enough data
    lag_features_added = []
    if len(train_processed) > 20:  # Only add lag features if we have sufficient data
        try:
            for store_dept, group in train_processed.groupby(['Store', 'Dept']):
                group_idx = group.index
                for lag in [1, 2, 4]:  # Reduced lag periods
                    lag_col = f'Sales_lag_{lag}'
                    if lag_col not in train_processed.columns:
                        train_processed[lag_col] = 0.0
                        lag_features_added.append(lag_col)

                    if len(group) > lag:
                        shifted_values = group['Weekly_Sales'].shift(lag)
                        train_processed.loc[group_idx, lag_col] = shifted_values

            print(f"Added lag features: {lag_features_added}")

        except Exception as e:
            print(f"Warning: Could not add lag features: {e}")
            lag_features_added = []

    # Update feature columns to include lag features
    feature_cols = [col for col in train_processed.columns
                   if col not in exclude_cols and
                   train_processed[col].dtype in ['int64', 'float64', 'int32', 'float32']]

    # Fill NaN values
    train_processed[feature_cols] = train_processed[feature_cols].fillna(0)

    # Apply same column structure to val and test datasets
    for col in feature_cols:
        if col not in val_processed.columns:
            val_processed[col] = 0.0
        if col not in test_processed.columns:
            test_processed[col] = 0.0

    # Ensure all datasets have the same columns and fill NaN
    val_processed[feature_cols] = val_processed[feature_cols].fillna(0)
    test_processed[feature_cols] = test_processed[feature_cols].fillna(0)

    # Make sure all datasets have the same feature columns
    common_cols = list(set(feature_cols) & set(val_processed.columns) & set(test_processed.columns))
    feature_cols = [col for col in feature_cols if col in common_cols]

    print(f"Final feature columns ({len(feature_cols)}): {feature_cols}")

    # Robust scaling
    scaler_features = StandardScaler()
    scaler_target = StandardScaler()

    # Fit scalers on training data
    train_features = train_processed[feature_cols].astype(np.float32)
    train_target = train_processed['Weekly_Sales'].values.reshape(-1, 1).astype(np.float32)

    scaler_features.fit(train_features)
    scaler_target.fit(train_target)

    # Transform all datasets
    train_processed[feature_cols] = scaler_features.transform(train_features)
    train_processed['Weekly_Sales'] = scaler_target.transform(train_target).flatten()

    val_features = val_processed[feature_cols].astype(np.float32)
    val_target = val_processed['Weekly_Sales'].values.reshape(-1, 1).astype(np.float32)
    val_processed[feature_cols] = scaler_features.transform(val_features)
    val_processed['Weekly_Sales'] = scaler_target.transform(val_target).flatten()

    test_features = test_processed[feature_cols].astype(np.float32)
    test_target = test_processed['Weekly_Sales'].values.reshape(-1, 1).astype(np.float32)
    test_processed[feature_cols] = scaler_features.transform(test_features)
    test_processed['Weekly_Sales'] = scaler_target.transform(test_target).flatten()

    # Create sequences
    X_train, y_train = create_sequences(train_processed, lookback, horizon)
    X_val, y_val = create_sequences(val_processed, lookback, horizon)
    X_test, y_test = create_sequences(test_processed, lookback, horizon)

    return (X_train, y_train, X_val, y_val, X_test, y_test,
            scaler_features, scaler_target)

# Usage in main execution:
if __name__ == "__main__":
    # Initialize WandB with optimized config
    wandb.init(
        project="Store-Sales-Forecasting-Optimized",
        entity="agasi22-free-university-of-tbilisi-",
        name="n-beats-optimized-run",
        config=wandb_config,
        tags=["nbeats", "optimized", "time-series", "walmart"]
    )

    config = wandb.config

    # Use optimized data preparation
    (X_train, y_train, X_val, y_val, X_test, y_test,
     scaler_features, scaler_target) = prepare_walmart_data_optimized(
        train_data, val_data, test_data,
        lookback=config.lookback, horizon=config.horizon
    )

    # Train optimized model
    model, history = train_nbeats_model_optimized(
        X_train, y_train, X_val, y_val,
        epochs=config.epochs,
        batch_size=config.batch_size,
        learning_rate=config.learning_rate
    )

Sorting by columns: ['Store', 'Dept', 'Week']
Initial feature columns: ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'IsHoliday', 'Year', 'Month', 'Week', 'Day', 'DayOfWeek', 'DayOfYear']
Added lag features: ['Sales_lag_1', 'Sales_lag_2', 'Sales_lag_4']
Final feature columns (23): ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'IsHoliday', 'Year', 'Month', 'Week', 'Day', 'DayOfWeek', 'DayOfYear', 'Sales_lag_1', 'Sales_lag_2', 'Sales_lag_4']
Creating sequences with 23 features: ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'IsHoliday', 'Year', 'Month', 'Week', 'Day', 'DayOfWeek', 'DayOfYear', 'Sales_lag_1', 'Sales_lag_2', 'Sales_lag_4']
Created 243766 sequences with shape: (2437

Model: "walmart_nbeats_optimized"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 16, 23)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 368)       │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_preprocess_1  │ (None, 256)       │     94,464 │ flatten[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_1                │ (None, 256)       │      1,024 │ dense_preprocess… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ bn_1[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_preprocess_2  │ (None, 128)       │     32,896 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_2                │ (None, 128)       │        512 │ dense_preprocess… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128)       │          0 │ bn_2[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_dense_1     │ (None, 128)       │     16,512 │ dropout_2[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_bn_1        │ (None, 128)       │        512 │ block_0_dense_1[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_dropout_1   │ (None, 128)       │          0 │ block_0_bn_1[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_dense_2     │ (None, 128)       │     16,512 │ block_0_dropout_… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_bn_2        │ (None, 128)       │        512 │ block_0_dense_2[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_dropout_2   │ (None, 128)       │          0 │ block_0_bn_2[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_dense_3     │ (None, 64)        │      8,256 │ block_0_dropout_… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_bn_3        │ (None, 64)        │        256 │ block_0_dense_3[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residual_proj_0     │ (None, 64)        │      8,256 │ dropout_2[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backcast_0 (Dense)  │ (None, 64)        │      4,160 │ block_0_bn_3[0][

 Total params: 218,467 (853.39 KB)

 Trainable params: 216,419 (845.39 KB)

 Non-trainable params: 2,048 (8.00 KB)

Epoch 1/100
3807/3809 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3627 - mae: 0.3432 - mape: 422.1582

3809/3809 ━━━━━━━━━━━━━━━━━━━━ 60s 13ms/step - loss: 0.3626 - mae: 0.3431 - mape: 422.0446 - val_loss: 1.3099 - val_mae: 0.6493 - val_mape: 444.0480 - learning_rate: 5.0000e-04
Epoch 2/100
3804/3809 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1429 - mae: 0.1956 - mape: 187.0859

3809/3809 ━━━━━━━━━━━━━━━━━━━━ 52s 14ms/step - loss: 0.1429 - mae: 0.1955 - mape: 187.0470 - val_loss: 1.0843 - val_mae: 0.6147 - val_mape: 325.6946 - learning_rate: 4.7500e-04
Epoch 3/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 79s 13ms/step - loss: 0.1259 - mae: 0.1791 - mape: 168.9199 - val_loss: 1.3731 - val_mae: 0.8129 - val_mape: 450.3263 - learning_rate: 4.5125e-04
Epoch 4/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 81s 13ms/step - loss: 0.1165 - mae: 0.1710 - mape: 204.6641 - val_loss: 1.7233 - val_mae: 0.9481 - val_mape: 502.6127 - learning_rate: 4.2869e-04
Epoch 5/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 85s 14ms/step - loss: 0.1116 - mae: 0.1660 - mape: 176.8219 - val_loss: 1.3770 - val_mae: 0.8864 - val_mape: 468.0576 - learning_rate: 4.0725e-04
Epoch 6/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 48s 13ms/step - loss: 0.1074 - mae: 0.1610 - mape: 153.4102 - val_loss: 1.4782 - val_mae: 0.9255 - val_mape: 503.1939 - learning_rate: 3.8689e-04
Epoch 7/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - loss: 0.